In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

import os
import wandb
from dotenv import load_dotenv

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score

import joblib
import pickle

from sklearn.impute import SimpleImputer

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures,
    FunctionTransformer,
)

# Definamos el `random_state`:
random_state = 42

# Cambiemos la fuente de las gráficas de matplotlib:
plt.rc('font', family='serif', size=12)

In [6]:
!pwd

/teamspace/studios/this_studio/mlprojectcars/notebooks


In [5]:
cd mlprojectcars/notebooks/

/teamspace/studios/this_studio/mlprojectcars/notebooks


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
#Data ingestion
DATASET_LOC = '../data/'
file0 = 'test_file.csv'
#file1 = 'data1.csv'

df0 = pd.read_csv(os.path.join(DATASET_LOC,file0))
#df1 = pd.read_csv(os.path.join(DATASET_LOC,file1))
df = pd.concat([df0])
#df = pd.concat([df0,df1])
df.sample(10)

,brand_model,year,price,transmission,mileage,fuelType,tax,milesPerGallon,engineSize
5892,Ford_KA,2015,4745,Manual,17328,Petrol,30,57.7,1.2
4907,Ford_Fiesta,2014,8399,Manual,65929,Petrol,145,47.9,1.6
2194,VW_Golf,2015,10995,Manual,28076,Petrol,30,53.3,1.4
1882,VW_Golf,2020,19990,Manual,2500,Diesel,145,57.7,1.6
2407,VW_T-Roc,2019,19892,Manual,9500,Petrol,150,42.2,1.5
5673,Ford_Streetka,2006,1850,Manual,74055,Petrol,290,35.8,1.6
2772,VW_Polo,2016,9498,Manual,15115,Petrol,20,60.1,1.2
6574,VW_Polo,2014,6984,Manual,57573,Petrol,125,51.4,1.2
2785,Ford_Focus,2018,13450,Manual,19729,Petrol,145,57.7,1.0
883,Ford_Kuga,2017,13995,Automatic,20893,Petrol,235,38.2,1.5


In [9]:
#Load preparation parameters
class MergeElectricToOther(BaseEstimator, TransformerMixin):
    def __init__(self, column_name):
        self.column_name = column_name

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        X_copy[self.column_name] = X_copy[self.column_name].replace('Electric', 'Other')
        return X_copy
        
preparation_file = os.path.join(DATASET_LOC,'preparation', 'preparation_params.pkl') #Parámetros
prep_params = joblib.load(preparation_file)

#Load preprocessing and model pipelines
preprocessor_file = os.path.join(DATASET_LOC,'model', 'preprocessor.pkl') #Parámetros
preprocessor = joblib.load(preprocessor_file)

model_file = os.path.join(DATASET_LOC,'model', 'model.pkl') #Parámetros
model = joblib.load(model_file)

In [10]:
test_df = df.drop(columns=prep_params['columns_to_drop'])
# Threshold to remove samples having missing values greater than threshold
max_nulls_allowed = prep_params['max_nulls_allowed']
# Getting Missing count of each sample            
nulls_per_row = test_df.isnull().sum(axis=1)
# Filter out rows with more than the allowed number of null values
test_df = test_df[nulls_per_row <= max_nulls_allowed]
# Eliminemos los registros con tamaño del motor menor a 1 litro:
test_df.drop(index=test_df[test_df.engineSize < prep_params["min_engineSize"]].index, inplace=True)
# Eliminemos el registro con año mayor que 2020:
test_df.drop(index=test_df[test_df['year'] > prep_params["max_year"]].index, inplace=True)
# Apliquemos el pipeline para combinar las columnas
test_df_transformed = prep_params["pipeline"].transform(test_df)

In [11]:
test_df_transformed

,brand_model,year,price,transmission,mileage,fuelType,tax,milesPerGallon,engineSize
0,Ford_Focus,2017,11498,Manual,18150,Petrol,20,60.1,1.0
1,VW_Touareg,2019,40995,Semi-Auto,1000,Diesel,145,34.5,3.0
2,VW_Tiguan,2017,23495,Semi-Auto,14839,Diesel,145,57.6,2.0
3,VW_Touareg,2018,37381,Semi-Auto,19718,Diesel,145,34.0,3.0
4,VW_Up,2017,7488,Manual,28449,Petrol,150,64.2,1.0
...,...,...,...,...,...,...,...,...,...
6620,VW_Golf,2019,20980,Semi-Auto,5398,Petrol,145,43.5,1.5
6621,Ford_Focus,2019,17699,Manual,5443,Petrol,145,60.1,1.0
6622,VW_Golf,2019,23940,Automatic,9535,Diesel,145,49.6,2.0
6623,Ford_EcoSport,2018,14500,Manual,15644,Petrol,145,54.3,1.0


In [16]:
X_test = test_df_transformed[preprocessor["cols"]]
y_test = test_df_transformed["price"]

In [17]:
X_test

,year,mileage,tax,milesPerGallon,engineSize,brand_model,transmission,fuelType
0,2017,18150,20,60.1,1.0,Ford_Focus,Manual,Petrol
1,2019,1000,145,34.5,3.0,VW_Touareg,Semi-Auto,Diesel
2,2017,14839,145,57.6,2.0,VW_Tiguan,Semi-Auto,Diesel
3,2018,19718,145,34.0,3.0,VW_Touareg,Semi-Auto,Diesel
4,2017,28449,150,64.2,1.0,VW_Up,Manual,Petrol
...,...,...,...,...,...,...,...,...
6620,2019,5398,145,43.5,1.5,VW_Golf,Semi-Auto,Petrol
6621,2019,5443,145,60.1,1.0,Ford_Focus,Manual,Petrol
6622,2019,9535,145,49.6,2.0,VW_Golf,Automatic,Diesel
6623,2018,15644,145,54.3,1.0,Ford_EcoSport,Manual,Petrol


In [18]:
X_test = preprocessor["preprocessor"].transform(X_test)

In [19]:
print("Model: Gradient boosting")
print(f'R^2: {model.score(X_test, y_test)}')
print(f'MAE: {mean_absolute_error(y_test, model.predict(X_test))}')
print('\n')

Model: Gradient boosting
R^2: 0.9699334356710758
MAE: 798.6473551525968




In [20]:
df = pd.DataFrame({'y_test':y_test, 'y_pred':model.predict(X_test)})
df

,y_test,y_pred
0,11498,10757.904938
1,40995,41974.687354
2,23495,22082.550870
3,37381,35503.576731
4,7488,7233.115601
...,...,...
6620,20980,21278.574743
6621,17699,16835.172585
6622,23940,25101.448791
6623,14500,13583.196652
